Teniendo en cuenta los resultados y las conclusiones de los anteriores notebooks, normalizaremos las variables Tempo, Loudness y Duration_ms y transformaremos a logaritmo el target Streams


In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv('spotify_limpio.csv')

df.head()

,Artist,Track,Album,Album_type,Stream,Danceability,Energy,Key,Key_name,Loudness,Speechiness,Acousticness,Liveness,Valence,Tempo,Tempo_category,Duration_ms,Licensed,official_video
0,Cults,Gilded Lily,Offering,album,83194819.0,0.400,0.571,11,B,-6.534,0.0288,0.4680,0.5190,0.146,61.859,Adagio,212736.0,True,True
1,Los Pericos,Runaway,Pericos & Friends,album,101408935.0,0.676,0.731,9,A,-5.253,0.0368,0.0438,0.6240,0.812,150.151,Allegro,233268.0,False,False
2,Richard Marx,Satisfied,Repeat Offender,album,4281404.0,0.572,0.914,8,G# / Ab,-8.436,0.0311,0.0717,0.3190,0.843,108.991,Andante,254467.0,False,True
3,The Supremes,Come See About Me,Where Did Our Love Go,album,70357721.0,0.732,0.506,7,G,-11.735,0.0462,0.6880,0.1460,0.867,126.325,Allegro,163093.0,False,False
4,El Fantasma,Equipo RR,Equipo RR,single,13232046.0,0.755,0.550,7,G,-6.102,0.1070,0.6180,0.0868,0.926,114.501,Moderato / Allegretto,159976.0,False,False


In [21]:
df = df.drop(columns=[ 'Artist', 'Track', 'Album', 'Album_type', 'Key_name', 'Tempo_category'])

In [22]:
df["log_stream"] = np.log1p(df["Stream"])

In [23]:
df.head()

,Stream,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Liveness,Valence,Tempo,Duration_ms,Licensed,official_video,log_stream
0,83194819.0,0.400,0.571,11,-6.534,0.0288,0.4680,0.5190,0.146,61.859,212736.0,True,True,18.236696
1,101408935.0,0.676,0.731,9,-5.253,0.0368,0.0438,0.6240,0.812,150.151,233268.0,False,False,18.434672
2,4281404.0,0.572,0.914,8,-8.436,0.0311,0.0717,0.3190,0.843,108.991,254467.0,False,True,15.269792
3,70357721.0,0.732,0.506,7,-11.735,0.0462,0.6880,0.1460,0.867,126.325,163093.0,False,False,18.069103
4,13232046.0,0.755,0.550,7,-6.102,0.1070,0.6180,0.0868,0.926,114.501,159976.0,False,False,16.398152


In [24]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

cols_to_scale = ["Loudness", "Tempo", "Duration_ms"]

df[cols_to_scale] = scaler.fit_transform(
    df[cols_to_scale]
)

Random forest ha sido el algoritmo seleccionado, porque nos daba las mejores métricas

In [25]:
import pandas as pd

X_train = df.drop(columns=['Stream', 'log_stream'], axis=1)
y_train = df[ 'log_stream']


Aqui estamos probando el algoritmo GridSearchCV para encontrar los mejores hiperparámetros

In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Modelo base
rf = RandomForestRegressor(
    random_state=42,
    n_jobs=-1
)

# Espacio de hiperparámetros
param_grid = {
    "n_estimators": np.arange(100, 600, 100),
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10]
}

# Grid Search + validación cruzada
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)

# Entrenamiento
grid_search.fit(X_train, y_train)

# Mejor modelo
best_rf = grid_search.best_estimator_

print("Mejores hiperparámetros:")
print(grid_search.best_params_)

print(f"Mejor RMSE CV: {-grid_search.best_score_:.4f}")

KeyboardInterrupt: 

A continuación vamos a usar el código RandomizedSearchCV para ver los mejores hiperparámetros, para tener otra alternativa y compararlos

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

rf = RandomForestRegressor(
    random_state=42,
    n_jobs=-1
)

param_dist = {
    "n_estimators": np.arange(100, 600, 100),
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10]
}

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=30,
    cv=5,
    scoring="neg_root_mean_squared_error",
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

best_rf = random_search.best_estimator_

print("Mejores hiperparámetros:")
print(random_search.best_params_)

print(f"Mejor RMSE CV: {-random_search.best_score_:.4f}")

/opt/anaconda3/envs/aprendizajeautomaticocd/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Mejores hiperparámetros:
{'n_estimators': np.int64(500), 'min_samples_split': 2, 'max_depth': None}
Mejor RMSE CV: 1.5060


Ya tenemos los mejores hiperparámetros, a continuación aplicamos el algoritmo Random forest, a todo train

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np


def val_cruz(model, X, y, n_splits=5, random_state=42):
    

    #cv = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)


    scores_rmse = cross_val_score(
        model,
        X,
        y,
        cv=n_splits,
        scoring="neg_root_mean_squared_error"
    )


    scores_mape = cross_val_score(
            model,
            X,
            y,
            cv=n_splits,
            scoring="neg_mean_absolute_percentage_error"
        )
        
    scores_mae = cross_val_score(
        model,
        X,
        y,
        cv=n_splits,
        scoring="neg_mean_absolute_error"
    )

    scores_r2 = cross_val_score(
        model,
        X,
        y,
        cv=n_splits,
        scoring="r2"
    )

    # Pasar a positivo
    scores_rmse = -scores_rmse
    scores_mae = -scores_mae
    scores_mape = -scores_mape 

    print(" Resultados validación cruzada")
    print("-" * 40)
    print(f"RMSE: {scores_rmse}")
    print(f"MAPE: {scores_mape}")
    print(f"MAE:  {scores_mae}")
    print(f"R²:   {scores_r2}")
    print("-" * 40)
    print(f"RMSE medio: {scores_rmse.mean():.4f} ± {scores_rmse.std():.4f}")
    print(f"MAPE medio:  {scores_mape.mean():.4f} ± {scores_mape.std():.4f}")
    print(f"MAE medio:  {scores_mae.mean():.4f} ± {scores_mae.std():.4f}")
    print(f"R² medio:   {scores_r2.mean():.4f} ± {scores_r2.std():.4f}")

    return scores_rmse.mean(), scores_rmse.std(), scores_mape.mean(), scores_mape.std(), scores_mae.mean(),scores_mae.std(),scores_r2.mean(),scores_r2.std()



Random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor


model = RandomForestRegressor(n_estimators = 500, max_depth = None, min_samples_split = 2, random_state=42)

model.fit(X_train, y_train)




,n_estimators,500
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False
